In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Fri Apr 30 20:25:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install emoji

     |████████████████████████████████| 133kB 3.9MB/s 


In [7]:
import StockSentimentDataset
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight

train_ds, val_ds, test_ds, embeddings, featurizer = StockSentimentDataset.load_all_data()

class_weights = compute_class_weight(class_weight='balanced', classes=[0,1,2], y=train_ds.labels)
class_weights_tensor = torch.FloatTensor(class_weights)

In [104]:
# import torch.nn.functional as f
from torch import nn

class LSTM(nn.Module):
 
  def __init__(self,  vocab_size, DIM_EMB=10, DIM_HID=10):
    super(LSTM, self).__init__()
    NUM_TAGS = 3
    (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
    self.embeddings = nn.Embedding(vocab_size, DIM_EMB, padding_idx = 0)
    self.lstm = nn.LSTM(DIM_EMB, DIM_HID, bidirectional = False, batch_first = True)
    self.dropout = nn.Dropout()
    self.linear = nn.Linear(DIM_HID, NUM_TAGS)
    self.logsoftmax = nn.LogSoftmax(dim = 2)

  def forward(self, X, train=False):
    embedded = self.embeddings(X)
    out, hidden = self.lstm(embedded)
    output = self.linear(out)
    output = self.logsoftmax(output)
    return output




In [106]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random
from pretrained_embeddings import add_embeddings_to_embedding_matrix

def train(train_ds, lstm):
  
  optimizer = optim.SGD(lstm.parameters(), lr=0.01)
  batchSize = 10
  train_dl = DataLoader(dataset=train_ds, batch_size=len(train_ds.samples),
    shuffle=True, collate_fn=StockSentimentDataset.collate_fn)
  nEpochs = 10
  for epoch in range(nEpochs):
    totalLoss = 0.0
    for x, y in tqdm.notebook.tqdm(train_dl, leave=False):
      lstm.zero_grad()
      x = x.cuda()
      y = y.cuda()
      y = torch.nn.functional.one_hot(y, 3)
      probs = lstm.forward(x)
      loss = (torch.neg(probs * y)).sum()

      totalLoss += loss
      loss.backward()
      optimizer.step()
    print(f"loss on epoch {epoch} = {totalLoss}")


lstm = LSTM(featurizer.vocab_size, DIM_HID=500, DIM_EMB=300).cuda()
train(train_ds, lstm)

loss on epoch 0 = 77870.0390625


loss on epoch 1 = 4416400.5


loss on epoch 2 = 1388414208.0


loss on epoch 3 = 2508930560.0


loss on epoch 4 = 1246757760.0


loss on epoch 5 = 238880144.0


loss on epoch 6 = nan


loss on epoch 7 = nan


loss on epoch 8 = nan


loss on epoch 9 = nan
